In [1]:
import random

from cachetools import cached, LFUCache, LRUCache, FIFOCache, RRCache

from mlfucache import MLFUCache
from benchmark.beladycache import BeladyCache

In [6]:
n = 100000
cbase = n // 100
size = cbase // 20

In [7]:
s = 1.1
HNs = sum([k ** -s for k in range(1, cbase + 1)])
def zipf_pmf(k, s):
    return sum([k ** -s / HNs])

p=0.01
def geometric_pmf(k, p):
    assert 0 < p <=1
    return (1 - p) ** (k - 1) * p


keys = range(1, cbase + 1)


In [8]:
class MyCache(MLFUCache):
    def popitem(self):
        key, value = super().popitem()
        print('Key "%s" evicted with value "%s"' % (key, value))
        return key, value


In [9]:
cinfo = {}
for distr, weights in {'zipf': [zipf_pmf(k, s) for k in keys], 'uniform': [1] * cbase,
                       'geometric': [geometric_pmf(k, p) for k in keys]}.items():
    requests = random.choices(keys, weights=weights, k=n)
    for cache in [RRCache(maxsize=size), FIFOCache(maxsize=size), LRUCache(maxsize=size), LFUCache(maxsize=size),
                  MLFUCache(maxsize=size), BeladyCache(maxsize=size, future=requests)]:

        @cached(cache=cache, key=lambda k: k, info=True)
        def storage(key) -> None:
            return None


        for key in requests:
            storage(key)

        cinfo[distr, cache.__class__.__name__] = storage.cache_info()
        print(f"{distr} - {cache.__class__.__name__}, {storage.cache_info().hits / n * 100:.2f}%")


zipf - RRCache, 50.95%
zipf - FIFOCache, 50.83%
zipf - LRUCache, 56.81%
zipf - LFUCache, 65.63%
zipf - MLFUCache, 68.34%
zipf - BeladyCache, 65.70%
uniform - RRCache, 4.97%
uniform - FIFOCache, 4.90%
uniform - LRUCache, 4.90%
uniform - LFUCache, 4.98%
uniform - MLFUCache, 4.96%
uniform - BeladyCache, 5.15%
geometric - RRCache, 22.87%
geometric - FIFOCache, 22.75%
geometric - LRUCache, 23.57%
geometric - LFUCache, 29.43%
geometric - MLFUCache, 37.41%
geometric - BeladyCache, 30.70%
